## Import

In [1]:
import os
import cv2
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
from torch.utils.tensorboard import SummaryWriter
# !pip install segmentation_models_pytorch
from tqdm import tqdm
import albumentations as A
from albumentations.pytorch import ToTensorV2
# from google.colab import drive
# !pip install transformers

In [2]:
# drive.mount('/content/drive')
from transformers import SamModel, SamProcessor, SamImageProcessor
import segmentation_models_pytorch as smp
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
# !pip install monai

c:\Users\user\anaconda3\envs\DeMa\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


cuda


In [3]:
# cd drive/MyDrive/building_segmentation

In [4]:
# RLE 디코딩 함수
def rle_decode(mask_rle, shape):
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape)

# RLE 인코딩 함수
def rle_encode(mask):
    pixels = mask.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

In [5]:
import matplotlib.pyplot as plt
import numpy as np

def show_mask(mask, ax, random_color=False):
    if random_color:
        color = np.concatenate([np.random.random(3), np.array([0.6])], axis=0)
    else:
        color = np.array([30/255, 144/255, 255/255, 0.6])
    h, w = mask.shape[-2:]
    mask_image = mask.reshape(h, w, 1) * color.reshape(1, 1, -1)
    ax.imshow(mask_image)

    fig, axes = plt.subplots()

    axes.imshow(np.array(image))
    ground_truth_seg = np.array(example["label"])
    show_mask(ground_truth_seg, axes)
    axes.title.set_text(f"Ground truth mask")
    axes.axis("off")

In [6]:
def create_prompt(ground_truth_mask):
    y_indices, x_indices = np.where(ground_truth_mask == 1)
    x_min, x_max = np.min(x_indices), np.max(x_indices)
    y_min, y_max = np.min(y_indices), np.max(y_indices)
    # add perturbation to bounding box coordinates
    H, W = ground_truth_mask.shape
    x_min = max(0, x_min - np.random.randint(0, 20))
    x_max = min(W, x_max + np.random.randint(0, 20))
    y_min = max(0, y_min - np.random.randint(0, 20))
    y_max = min(H, y_max + np.random.randint(0, 20))
    bbox = [x_min, y_min, x_max, y_max]

    return bbox

## Utils

In [7]:
class SatelliteDataset(Dataset):
    def __init__(self, csv_file, processor=None, transform = None, infer=False):
        self.data = pd.read_csv(csv_file)
        self.processor = processor
        self.infer = infer
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path = self.data.iloc[idx, 1]
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        if self.infer:
            if self.transform:
                resize = self.transform(image = image)
                image = resize["image"]
                bbox = [0, 0, image.shape[0], image.shape[1]]
                augmented = self.processor(image, input_boxes=[[bbox]], return_tensors="pt")
                augmented = {k:v.squeeze(0) for k,v in augmented.items()}
            return augmented

        mask_rle = self.data.iloc[idx, 2]
        mask = rle_decode(mask_rle, (image.shape[0], image.shape[1]))
        bbox = create_prompt(mask)

        resize = self.transform(image = image, mask = mask)
        mask = resize["mask"]

        augmented = self.processor(image, input_boxes=[[bbox]], return_tensors="pt")
        augmented = {k:v.squeeze(0) for k,v in augmented.items()}
        augmented["mask"] = mask

        return augmented

## Custom Dataset

In [8]:
from transformers import SamImageProcessor

processor = SamProcessor.from_pretrained("facebook/sam-vit-base")
transform = A.Compose(
    [
        A.Resize(256, 256),
        ToTensorV2()
    ]
)

test_transform = A.Compose(
    [
        A.Resize(1024, 1024),
        ToTensorV2()
    ]
)
train_csv_path = "train.csv"
dataset = SatelliteDataset(train_csv_path, processor=processor, transform = transform)
train_dataset_length = int(len(dataset) * 0.9)
validation_dataset_length = int(len(dataset) * 0.1)

train_dataset, validation_dataset = random_split(dataset, [train_dataset_length, validation_dataset_length])

train_dataloader = DataLoader(train_dataset, batch_size=4, shuffle=True)
validation_dataloader = DataLoader(validation_dataset, batch_size=4, shuffle=True)

## Define Model

In [9]:
model = SamModel.from_pretrained("facebook/sam-vit-base")
model.load_state_dict(torch.load("ep2.pt"))
# make sure we only compute gradients for mask decoder
for name, param in model.named_parameters():
  if name.startswith("vision_encoder") or name.startswith("prompt_encoder"):
    param.requires_grad_(False)

model.to(device)

SamModel(
  (shared_image_embedding): SamPositionalEmbedding()
  (vision_encoder): SamVisionEncoder(
    (patch_embed): SamPatchEmbeddings(
      (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
    )
    (layers): ModuleList(
      (0-11): 12 x SamVisionLayer(
        (layer_norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (attn): SamVisionAttention(
          (qkv): Linear(in_features=768, out_features=2304, bias=True)
          (proj): Linear(in_features=768, out_features=768, bias=True)
        )
        (layer_norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (mlp): SamMLPBlock(
          (lin1): Linear(in_features=768, out_features=3072, bias=True)
          (lin2): Linear(in_features=3072, out_features=768, bias=True)
          (act): GELUActivation()
        )
      )
    )
    (neck): SamVisionNeck(
      (conv1): Conv2d(768, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (layer_norm1): SamLayerNorm()
     

In [10]:
import monai
seg_loss = monai.losses.DiceCELoss(sigmoid=True, squared_pred=True, reduction='mean')
# loss function과 optimizer 정의
criterion = torch.nn.BCEWithLogitsLoss()
Dice_Loss = smp.losses.DiceLoss(mode='binary')
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

In [11]:
import monai
seg_loss = monai.losses.DiceCELoss(sigmoid=True, squared_pred=True, reduction='mean')
# loss function과 optimizer 정의
criterion = torch.nn.BCEWithLogitsLoss()
Dice_Loss = smp.losses.DiceLoss(mode='binary')
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

# training loop
save_intv = 2
model.train()
for epoch in range(2, 6):  # 15 에폭 동안 학습합니다.
    train_loss = 0
    for input in tqdm(train_dataloader):
        masks = input["mask"].float().to(device)

        optimizer.zero_grad()
        outputs = model(pixel_values=input["pixel_values"].to(device),
                      input_boxes=input["input_boxes"].to(device),
                      multimask_output=False)

        predicted_masks = outputs.pred_masks.squeeze(1)
        loss = seg_loss(predicted_masks, masks.unsqueeze(1))
        score = 1 - loss.item()
        print("dice score result: ")
        print(score)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        torch.cuda.empty_cache()

    if (epoch+1) % save_intv == 0:
        torch.save(model.state_dict(), os.path.join("weights", f'ep{epoch+1}.pt'))

    val_loss = 0
    with torch.no_grad():
        model.eval()
        for val_input in tqdm(validation_dataloader):
            masks = val_input["mask"].float().to(device)

            optimizer.zero_grad()
            outputs = model(pixel_values=val_input["pixel_values"].to(device),
                      input_boxes=val_input["input_boxes"].to(device),
                      multimask_output=False)

            validation_predicted_masks = outputs.pred_masks.squeeze(1)
            validation_loss = seg_loss(validation_predicted_masks, masks.unsqueeze(1))
            score = 1- validation_loss.item()
            print(score)
            val_loss += validation_loss.item()
            torch.cuda.empty_cache()


    print(f'Epoch {epoch+1}, Loss: {train_loss/len(train_dataloader)}')
    print(f'Epoch {epoch+1}, Loss: {val_loss/len(validation_dataloader)}')

  0%|          | 0/1607 [00:00<?, ?it/s]

dice score result: 
0.7562186270952225


  0%|          | 2/1607 [00:06<1:25:28,  3.20s/it]

dice score result: 
0.7663793563842773


  0%|          | 3/1607 [00:08<1:12:42,  2.72s/it]

...

100%|█████████▉| 1606/1607 [1:02:50<00:02,  2.36s/it]

dice score result: 
0.8090013414621353


100%|██████████| 1607/1607 [1:02:51<00:00,  2.35s/it]

dice score result: 
0.7893102765083313


RuntimeError: Parent directory weights does not exist.

## Model Train

In [12]:
test_dataset = SatelliteDataset(csv_file="test.csv", processor=processor, transform=test_transform, infer=True)
test_dataloader = DataLoader(test_dataset, batch_size=2, shuffle=False)

In [13]:
gc.collect()
torch.cuda.empty_cache()

NameError: name 'gc' is not defined

## Inference

In [15]:
import os

# def check_and_reconnect_drive():
#     try:
#         # Check if Google Drive is still connected
#         os.listdir('/content/drive')
#     except:
#         # If not, reconnect it
#         from google.colab import drive
#         drive.mount('/content/drive', force_remount=True)

# Then call this function every so often in your main script
# check_and_reconnect_drive()

with torch.no_grad():
    model.eval()
    result = []
    for input in tqdm(test_dataloader):
        outputs = model(pixel_values=input["pixel_values"].to(device),
                      input_boxes=input["input_boxes"].to(device),
                      multimask_output=False)

        predicted_masks = outputs.pred_masks
        masks = torch.sigmoid(predicted_masks).cpu().numpy()
        masks = np.squeeze(masks, axis=1)
        masks = (masks > 0.35).astype(np.uint8) # Threshold = 0.35

        for i in range(len(input)):
            mask_rle = rle_encode(masks[i])
            if mask_rle == '': # 예측된 건물 픽셀이 아예 없는 경우 -1
                result.append(-1)
            else:
                result.append(mask_rle)
        check_and_reconnect_drive()
        torch.cuda.empty_cache()

  0%|          | 0/30320 [00:00<?, ?it/s]


IndexError: index 2 is out of bounds for axis 0 with size 2

#SAM 자체의 mask post processing 사용


In [16]:
import os, gc


with torch.no_grad():
    model.eval()
    result = []
    for input in tqdm(test_dataloader):
        outputs = model(pixel_values=input["pixel_values"].to(device),
                      input_boxes=input["input_boxes"].to(device),
                      multimask_output=False)

        predicted_masks = outputs.pred_masks
        #masks = torch.sigmoid(predicted_masks).cpu().numpy()
        #masks = np.squeeze(masks, axis=1)

        #masks = (masks > 0).astype(np.uint8) # Threshold = 0.35
        masks = SamImageProcessor().post_process_masks(masks = predicted_masks, original_sizes = [(224, 224), (224, 224), (224, 224), (224, 224)],
                                                       reshaped_input_sizes = [(256, 256), (256, 256), (256, 256), (256, 256)])
        masks0 = torch.tensor(masks[0])
        masks0.squeeze(0)
        masks1 = torch.tensor(masks[1])
        masks1.squeeze(0)
        masks2 = torch.tensor(masks[2])
        masks2.squeeze(0)
        masks3 = torch.tensor(masks[3])
        masks3.squeeze(0)

        masks = torch.cat([masks0,masks1,masks2,masks3], dim=0)
        masks = masks.cpu().numpy()
        for i in range(len(input)):
            mask_rle = rle_encode(masks[i])
            if mask_rle == '': # 예측된 건물 픽셀이 아예 없는 경우 -1
                result.append(-1)
            else:
                result.append(mask_rle)
        gc.collect()
        torch.cuda.empty_cache()

  0%|          | 0/30320 [00:00<?, ?it/s]


IndexError: index 2 is out of bounds for dimension 0 with size 2

In [ ]:
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
submit = pd.read_csv('./sample_submission.csv')
submit['mask_rle'] = result

## Submission

In [ ]:
submit.to_csv('./submit.csv', index=False)